In [20]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import cv2 as cv
import tensorflow as tf
from keras import backend
from keras.models import *
from keras.metrics import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [15]:
def resized(img_path):
  img = cv.imread(img_path)
  img = cv.resize(img,(256,256))
  img = img/255.0
  return img

In [17]:
ap = []
lat = []
ap_v = []
ap_s = []
ap_p = []
lat_v = []
lat_a = []
lat_d = []
lat_s = []
lat_p = []

textr = open("location_train.txt").read()
files = textr.split("\n")

for file in files:
  if("ap." in file.lower()):
    ap.append(file)
  elif("lat." in file.lower()):
    lat.append(file)
  elif("lat_a" in file.lower()):
    lat_a.append(file)
  elif("lat_v" in file.lower()):
    lat_v.append(file)
  elif("lat_p" in file.lower()):
    lat_p.append(file)
  elif("lat_d" in file.lower()):
    lat_d.append(file)
  elif("lat_s" in file.lower()):
    lat_s.append(file)
  elif("ap_v" in file.lower()):
    ap_v.append(file)
  elif("ap_s" in file.lower()):
    ap_s.append(file)
  elif("ap_p" in file.lower()):
    ap_p.append(file)

In [18]:
ap_seg = [ap_p,ap_s,ap_v] 
lat_seg = [lat_a,lat_d,lat_p,lat_s,lat_v]

AP = []
LAT = []

for i in range(3):
  X_train, X_valid, y_train, y_valid = train_test_split(ap,ap_seg[i], test_size = 0.2, random_state=42)
  AP.append([X_train, X_valid, y_train, y_valid])

for i in range(5):
  X_train, X_valid, y_train, y_valid = train_test_split(lat,lat_seg[i], test_size = 0.2, random_state=42)
  LAT.append([X_train, X_valid, y_train, y_valid])

In [19]:
def image_generator(X,y, batch_size):
  while True:
    batch  = np.random.choice(len(X),size = batch_size)
    batch_input  = []
    batch_output = []
    
    for i in batch:
      output = resized(y[i])
      input = resized(X[i])
      batch_input += [input]
      batch_output += [output]
      
    batch_x = np.array(batch_input)
    batch_y = np.array(batch_output)
    
    yield (batch_x, batch_y)

In [22]:
def conv_block(image,n_filters,size_kernel):
    out = Conv2D(filters = n_filters, kernel_size = (size_kernel,size_kernel), padding = 'same',kernel_initializer = 'he_normal')(image)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(filters = n_filters, kernel_size = (size_kernel,size_kernel), padding = 'same',kernel_initializer = 'he_normal')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    return out

def up_sampling(n_filters,dropout,convh,convd):
  up = Conv2DTranspose(n_filters, (3, 3), strides = (2, 2), padding = 'same')(convh)
  up = concatenate([up, convd])
  up = Dropout(rate=dropout)(up)
  conv = conv_block(up,n_filters,3)
  return conv

def unet(image,n_filters,kernel_size,dropout):    
  conv1 = conv_block(image,n_filters,kernel_size)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

  n_filters = n_filters*2
  conv2 = conv_block(pool1,n_filters,kernel_size) 
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

  n_filters = n_filters*2
  conv3 = conv_block(pool2,n_filters,kernel_size)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

  n_filters = n_filters*2
  conv4 = conv_block(pool3,n_filters,kernel_size)
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

  n_filters = n_filters*2
  conv5 = conv_block(pool4,n_filters,kernel_size)
  conv_seq = [conv4,conv3,conv2,conv1]
  out = conv5
 
  for convd in conv_seq:
    n_filters = int(n_filters/2)
    out = up_sampling(n_filters,dropout,convh=out,convd=convd)

  output = Conv2D(3, (1, 1), activation='sigmoid')(out)
  model = Model(inputs=[image], outputs=[output])
  return model

In [ ]:
input_img = Input((256,256,3))
filepath_ap=["ap_weights1.hdf5","ap_weights2.hdf5","ap_weights3.hdf5"]
filepath_lat=["lat_weights1.hdf5","lat_weights2.hdf5","lat_weights3.hdf5","lat_weights4.hdf5","lat_weights5.hdf5"]

model = unet(input_img, n_filters=16,kernel_size=3,dropout=0.1)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
checkpoint = ModelCheckpoint(filepath_ap[1], monitor='val_loss', verbose=1, save_best_only=True, mode='max')

results = model.fit_generator(image_generator(AP[1][0],AP[1][2],10),steps_per_epoch=56, epochs=20,validation_data=image_generator(AP[1][1],AP[1][3],10),validation_steps=6,callbacks = [checkpoint])

In [ ]:
model = unet(input_img, n_filters=16,kernel_size=3,dropout=0.1)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
checkpoint = ModelCheckpoint(filepath_lat[j], monitor='val_iou', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
results = model.fit_generator(image_generator(LAT[j][0],LAT[j][2],10),steps_per_epoch=50, epochs=20,validation_data=image_generator(LAT[j][1],LAT[j][3],10),validation_steps=12,callbacks = [callbacks_list])